## Django 사용하여 html에 출력

### view.py

In [ ]:
from .models import Nutrition, Foodimage, Foodlist
from user.models import *
from django.core.files.storage import FileSystemStorage
from django.http.response import HttpResponse
from django.shortcuts import render, redirect
from django.db.models import Q
import os, json, datetime
from django.db.models import Sum


# Create your views here.
def main(request):
    # try:
    #     if request.session['user']:
            if request.method == 'POST':
                mainTime = request.POST['mainDate']
            else:
                mainTime = datetime.datetime.now().strftime("%Y-%m-%d")
                # current = datetime.datetime.now().strftime("%Y-%m-%d")
            user = User.objects.get(user_name=request.session['user'])
            ## 총 섭취 칼로리
            BMI = user.userWeight / ((user.userHeight/100) ** 2) # BMI 
            # 과체중 이상 (BMI >= 25 : 과체중, BMI >= 30 비만)
            if(BMI >= 25): 
                std_weight = user.userWeight * 30 - 250
            # 과체중 이하
            else :
                std_weight = (user.userHeight-100)*0.9 * 30 # 표준체중 (편집됨) 

            ## 아침 식단
            Breakfast_final = []
            Breakfast_image_final = []
            Breakfast_images = Foodimage.objects.filter(Q(user_name = request.session['user']) & Q(eatDate=mainTime) & Q(eatTime='B')&Q(isDeleted=0))
            for Breakfast_image in Breakfast_images:
                Breakfast_list = Foodlist.objects.filter(fileSeq=Breakfast_image.fileSeq)
                Breakfast_image_final.append({"BreakfastImage" : Breakfast_image.foodImg})
                for Breakfast in Breakfast_list:
                    gram = float(Breakfast.food_id.에너지_field)*float(Breakfast.food_gram/float(Breakfast.food_id.number_1회_제공량_g_field))
                    Breakfast_final.append({"name": Breakfast.food_id.food_name, 'gram': Breakfast.food_gram, 'cal': round(gram,2)})

            ## 점심 식단
            Lunch_final = []
            Lunch_image_final = []
            Lunch_images = Foodimage.objects.filter(Q(user_name = request.session['user']) & Q(eatDate=mainTime) & Q(eatTime='L')&Q(isDeleted=0))
            for Lunch_image in Lunch_images:
                Lunch_list = Foodlist.objects.filter(fileSeq=Lunch_image.fileSeq)
                Lunch_image_final.append({"LunchImage" : Lunch_image.foodImg})
                for Lunch in Lunch_list:
                    gram = float(Lunch.food_id.에너지_field)*float(Lunch.food_gram/float(Lunch.food_id.number_1회_제공량_g_field))
                    Lunch_final.append({"name": Lunch.food_id.food_name, 'gram': Lunch.food_gram, 'cal': round(gram,2)})

            ## 저녁 식단
            Dinner_final = []
            Dinner_image_final = []
            Dinner_images = Foodimage.objects.filter(Q(user_name = request.session['user']) & Q(eatDate=mainTime) & Q(eatTime='D')&Q(isDeleted=0))
            for Dinner_image in Dinner_images:
                Dinner_list = Foodlist.objects.filter(fileSeq=Dinner_image.fileSeq)
                Dinner_image_final.append({"DinnerImage" : Dinner_image.foodImg})
                for Dinner in Dinner_list:
                    gram = float(Dinner.food_id.에너지_field)*float(Dinner.food_gram/float(Dinner.food_id.number_1회_제공량_g_field))
                    Dinner_final.append({"name": Dinner.food_id.food_name, 'gram': Dinner.food_gram, 'cal': round(gram,2)})

            ## 하루 섭취 영양 성분
            
            image_file = Foodimage.objects.filter(Q(user_name = request.session['user'])&Q(eatDate=mainTime)&Q(isDeleted=0))

            단백질=0; 탄수화물=0; 지방=0; 에너지=0
            for image in image_file:
                food_list = Foodlist.objects.filter(fileSeq=image.fileSeq)
                for food in food_list:
                    nutrition = Nutrition.objects.filter(food_id = food.food_id.food_id)#foodlist의 food_id 값을 가져왔음

                    섭취량 = nutrition[0].number_1회_제공량_g_field
                    단백질 += (float(nutrition[0].단백질_g_field) * (int(food.food_gram)/int(섭취량)))
                    탄수화물 += (float(nutrition[0].탄수화물_g_field) * (int(food.food_gram)/int(섭취량)))
                    지방 += (float(nutrition[0].지방_g_field) * (int(food.food_gram)/int(섭취량)))
                    에너지 += (float(nutrition[0].에너지_field) * (int(food.food_gram)/int(섭취량)))

 
            contents = {'기준에너지': std_weight, '에너지': round(에너지,2), '단백질' : round(단백질,2), '탄수화물': round(탄수화물,2), '지방' : round(지방, 2),
                         'mainTime': mainTime, 'Breakfast': Breakfast_final, 'Lunch' : Lunch_final, 'Dinner' : Dinner_final,
                         'BreakfastImage' : Breakfast_image_final, 'LunchImage' : Lunch_image_final, 'DinnerImage' : Dinner_image_final}
            return render(request, 'main/main.html',contents)

### urls.py

In [ ]:
from django.urls import path
from . import views

# app_name = 'main'

urlpatterns = [
    # path('', views.home, name='home'),
    # path('signin', views.signin, name='signin'),
    # path('signup', views.signup, name='signup'),
    path('', views.main, name="main"),
    path('search', views.search, name='search'),
    path('upload', views.upload, name='upload'),
    path('analyze', views.analyze, name='analyze'),
    path('warning', views.warning, name='warning')
]